# Generate:

## Imports

In [1]:
import tensorflow as tf
import os
import numpy as np
import midiutil
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from tensorflow import keras
from midiutil import MIDIFile
from os import listdir

## Load Models

In [2]:
m_n_path = os.path.join(os.getcwd(),'Models/4 Layers/m_n.h5')
m_e_path = os.path.join(os.getcwd(),'Models/4 Layers/m_e.h5')
m_h_path = os.path.join(os.getcwd(),'Models/4 Layers/m_h.h5')

m_n=keras.models.load_model(m_n_path)
m_e=keras.models.load_model(m_e_path)
m_h=keras.models.load_model(m_h_path)

## Load Input/Output

In [5]:
in_file_path= os.path.join(os.getcwd(), 'Data//Processed', 'input.txt')
out_file_path= os.path.join(os.getcwd(), 'Data//Processed', 'output.txt')

inp = pd.read_csv(in_file_path,sep=",",header=None)
out = pd.read_csv(out_file_path,sep=",",header=None)

out_save = out.copy()

In [6]:
out_fixed=out.copy()
out_fixed=out_fixed.astype(np.int)
out =pd.DataFrame(out_fixed)

In [7]:
count_notes = np.sum(out,axis=1)
count_notes_hot = pd.get_dummies(count_notes)
out_hot=pd.concat([out,count_notes_hot],axis=1,ignore_index=True)


## Predict

In [6]:
inp_test_path = os.path.join(os.getcwd(),'Data/Processed/Testing/input.txt')
inp_test=pd.read_csv(inp_test_path,sep=',',header=None)

y_pred_n = m_n.predict(inp_test,batch_size=8)
y_pred_e = m_e.predict(inp_test,batch_size=8)
y_pred_h = m_h.predict(inp_test,batch_size=8)

FileNotFoundError: File b'/afs/inf.ed.ac.uk/user/s17/s1709221/AMT/Automatic-Music-Transcription-master/Data/Processed/Testing/input.txt' does not exist

In [365]:
tbin_path = os.path.join(os.getcwd(),'Data/Processed/Testing/tbin.txt')
a=pd.read_csv(tbin_path,sep=',',header=None)
b=np.zeros(1)
tbin=np.append(b,a)

## Method 1

In [366]:
y_note_pred_n = np.zeros((y_pred_n.shape[0],y_pred_n.shape[1]))
for i in range(y_pred_n.shape[0]):
    a=np.zeros((88,1))
    a[:,0]=y_pred_n[i,:].copy()
    a[a<0.03]=0
    kmeans=KMeans(n_clusters=2).fit(a)
    threshold = np.maximum(kmeans.cluster_centers_[0],kmeans.cluster_centers_[1])
    if threshold==0:
        threshold = 1
    a=y_pred_n[i,0:88].copy()
    a[a>=threshold]=1
    a[a<threshold]=0
    y_note_pred_n[i,:]=a

In [367]:
track    = 0
channel  = 0
time     = 0    # In beats
duration = 1    # In beats
tempo    = 60   # In BPM
volume   = 100  # 0-127, as per the MIDI standard
MyMIDI_n = MIDIFile(1)
MyMIDI_n.addTempo(track, time, tempo)

In [368]:
b=np.zeros((1,3))
y_copy = y_note_pred_n.copy()
time=0
time_len = np.diff(tbin)[1]
for i in range(tbin.shape[0]):
    if (i==0):
        time=0
    elif (i==1):
        time = time+np.diff(tbin)[0]
    else:
        time=time+time_len
        
    a=np.where(y_copy[i,:]==1)[0]
    for j in range(a.shape[0]):
        b_temp=np.zeros((1,3))
        b_temp[0,0]=a[j]
        b_temp[0,1]=time
        k=0
        c=np.where(y_copy[i+k,:]==1)[0]
        if (i==tbin.shape[0]-1):
            break
        while (y_copy[i+k,a[j]]==1):
            if (i+k==tbin.shape[0]-1):
                break
            k=k+1
            b_temp[0,2]=b_temp[0,2]+1
        for l in range(k+1):
            y_copy[i+l,a[j]]=0
        b=np.append(b,b_temp,axis=0)
            

In [369]:
b[:,0]=b[:,0]+21
b=b[1:b.shape[0],:]
for i in range(b.shape[0]):
    MyMIDI_n.addNote(track,channel,b[i,0].astype(np.int),b[i,1],b[i,2]*time_len,volume)

In [370]:
with open("out_n.mid", "wb") as output_file:
    MyMIDI_n.writeFile(output_file)

## Method 2

In [371]:
y_note_pred_e=np.zeros((y_pred_e.shape[0],88))
for i in range(y_pred_e.shape[0]):
    a = y_pred_e[i,:].copy()
    if (np.argmax(a)==88):
        y_note_pred_e[i,:]=np.zeros(88)
    else:
        a=np.zeros((88,1))
        a[:,0]=y_pred_e[i,0:88].copy()
        a[a<0.03]=0
        kmeans1=KMeans(n_clusters=2).fit(a)
        threshold = np.maximum(kmeans1.cluster_centers_[0],kmeans1.cluster_centers_[1])
        if threshold==0:
            threshold = 1
        a=y_pred_e[i,0:88].copy()
        a[a>=threshold]=1
        a[a<threshold]=0
        y_note_pred_e[i,:]=a

In [372]:
track    = 0
channel  = 0
time     = 0    # In beats
duration = 1    # In beats
tempo    = 60   # In BPM
volume   = 100  # 0-127, as per the MIDI standard
MyMIDI_e = MIDIFile(1)
MyMIDI_e.addTempo(track, time, tempo)

In [373]:
b=np.zeros((1,3))
y_copy = y_note_pred_e.copy()
time=0
time_len = np.diff(tbin)[1]
for i in range(tbin.shape[0]):
    if (i==0):
        time=0
    elif (i==1):
        time = time+np.diff(tbin)[0]
    else:
        time=time+time_len
        
    a=np.where(y_copy[i,:]==1)[0]
    for j in range(a.shape[0]):
        b_temp=np.zeros((1,3))
        b_temp[0,0]=a[j]
        b_temp[0,1]=time
        k=0
        c=np.where(y_copy[i+k,:]==1)[0]
        if (i==tbin.shape[0]-1):
            break
        while (y_copy[i+k,a[j]]==1):
            if (i+k==tbin.shape[0]-1):
                break
            k=k+1
            b_temp[0,2]=b_temp[0,2]+1
        for l in range(k+1):
            y_copy[i+l,a[j]]=0
        b=np.append(b,b_temp,axis=0)

In [374]:
b[:,0]=b[:,0]+21
b=b[1:b.shape[0],:]
for i in range(b.shape[0]):
    MyMIDI_e.addNote(track,channel,b[i,0].astype(np.int),b[i,1],b[i,2]*time_len,volume)

In [375]:
with open("out_e.mid", "wb") as output_file:
    MyMIDI_e.writeFile(output_file)

## Method 3

In [376]:
threshold_pred_h = y_pred_h.copy()[:,0:88]
num_pred_prob = y_pred_h.copy()
for i in range(y_pred_h.shape[0]):
    a = num_pred_prob[:,88:num_pred_prob.shape[1]]
    num_pred=count_notes_hot.columns[np.argmax(a[i,:])]
    pred_idx = np.flip(np.argsort(threshold_pred_h[i,:]))[0:num_pred]
    b=np.zeros(88)
    if (num_pred==0):
        threshold_pred_h[i,:]=b
    else:
        b[pred_idx]=1
        threshold_pred_h[i,:]=b
        
y_note_pred_h = threshold_pred_h[:,0:88]

In [377]:
track    = 0
channel  = 0
time     = 0    # In beats
duration = 1    # In beats
tempo    = 60   # In BPM
volume   = 100  # 0-127, as per the MIDI standard
MyMIDI_h = MIDIFile(1)
MyMIDI_h.addTempo(track, time, tempo)

In [378]:
b=np.zeros((1,3))
y_copy = y_note_pred_h.copy()
time=0
time_len = np.diff(tbin)[1]
for i in range(tbin.shape[0]):
    if (i==0):
        time=0
    elif (i==1):
        time = time+np.diff(tbin)[0]
    else:
        time=time+time_len
        
    a=np.where(y_copy[i,:]==1)[0]
    for j in range(a.shape[0]):
        b_temp=np.zeros((1,3))
        b_temp[0,0]=a[j]
        b_temp[0,1]=time
        k=0
        c=np.where(y_copy[i+k,:]==1)[0]
        if (i==tbin.shape[0]-1):
            break
        while (y_copy[i+k,a[j]]==1):
            if (i+k==tbin.shape[0]-1):
                break
            k=k+1
            b_temp[0,2]=b_temp[0,2]+1
        for l in range(k+1):
            y_copy[i+l,a[j]]=0
        b=np.append(b,b_temp,axis=0)  

In [379]:
b[:,0]=b[:,0]+21
b=b[1:b.shape[0],:]
for i in range(b.shape[0]):
    MyMIDI_h.addNote(track,channel,b[i,0].astype(np.int),b[i,1],b[i,2]*time_len,volume)

In [380]:
with open("out_h.mid", "wb") as output_file:
    MyMIDI_h.writeFile(output_file)

## All Combined

In [3]:
def postprocess(y_pred,tbin,):
    b=np.zeros((1,3))
    y_copy = y_pred.copy()
    time=0
    time_len = np.diff(tbin)[1]
    for i in range(tbin.shape[0]):
        if (i==0):
            time=0
        elif (i==1):
            time = time+np.diff(tbin)[0]
        else:
            time=time+time_len
        
        a=np.where(y_copy[i,:]==1)[0]
        for j in range(a.shape[0]):
            b_temp=np.zeros((1,3))
            b_temp[0,0]=a[j]
            b_temp[0,1]=time
            k=0
            c=np.where(y_copy[i+k,:]==1)[0]
            if (i==tbin.shape[0]-1):
                break
            while (y_copy[i+k,a[j]]==1):
                if (i+k==tbin.shape[0]-1):
                    break
                k=k+1
                b_temp[0,2]=b_temp[0,2]+1
            for l in range(k+1):
                y_copy[i+l,a[j]]=0
            b=np.append(b,b_temp,axis=0)
    return b

In [8]:
input_dir_path_in=os.path.join(os.getcwd(),'Data/Processed/Testing/input')
tbin_dir_path_in=os.path.join(os.getcwd(),'Data/Processed/Testing/tbin')

input_file = listdir(input_dir_path_in)
tbin_files = listdir(tbin_dir_path_in)
for i in range(len(input_file)):
    inp_test_path = os.path.join(os.getcwd(),'Data/Processed/Testing/input',input_file[i])
    inp_test=pd.read_csv(inp_test_path,sep=',',header=None)

    y_pred_n = m_n.predict(inp_test,batch_size=8)
    y_pred_e = m_e.predict(inp_test,batch_size=8)
    y_pred_h = m_h.predict(inp_test,batch_size=8)
    
    tbin_path = os.path.join(os.getcwd(),'Data/Processed/Testing/tbin',tbin_files[i])
    a=pd.read_csv(tbin_path,sep=',',header=None)
    b=np.zeros(1)
    tbin=np.append(b,a)
    
    time_len = np.diff(tbin)[1]
    
    y_note_pred_n = np.zeros((y_pred_n.shape[0],y_pred_n.shape[1]))
    for j in range(y_pred_n.shape[0]):
        a=np.zeros((88,1))
        a[:,0]=y_pred_n[j,:].copy()
        a[a<0.03]=0
        kmeans=KMeans(n_clusters=2).fit(a)
        threshold = np.maximum(kmeans.cluster_centers_[0],kmeans.cluster_centers_[1])
        if threshold==0:
            threshold = 1
        a=y_pred_n[j,0:88].copy()
        a[a>=threshold]=1
        a[a<threshold]=0
        y_note_pred_n[j,:]=a
    
    track    = 0
    channel  = 0
    time     = 0    # In beats
    duration = 1    # In beats
    tempo    = 60   # In BPM
    volume   = 100  # 0-127, as per the MIDI standard
    MyMIDI_n = MIDIFile(1)
    MyMIDI_n.addTempo(track, time, tempo)
    
    b=postprocess(y_note_pred_n,tbin)
            
    b[:,0]=b[:,0]+21
    b=b[1:b.shape[0],:]
    for j in range(b.shape[0]):
        MyMIDI_n.addNote(track,channel,b[j,0].astype(np.int),b[j,1],b[j,2]*time_len,volume)
    
    with open("out_"+input_file[i][:-4]+"_n.mid", "wb") as output_file:
        MyMIDI_n.writeFile(output_file)
        
    y_note_pred_e=np.zeros((y_pred_e.shape[0],88))
    
    for j in range(y_pred_e.shape[0]):
        a = y_pred_e[j,:].copy()
        if (np.argmax(a)==88):
            y_note_pred_e[j,:]=np.zeros(88)
        else:
            a=np.zeros((88,1))
            a[:,0]=y_pred_e[j,0:88].copy()
            a[a<0.03]=0
            kmeans1=KMeans(n_clusters=2).fit(a)
            threshold = np.maximum(kmeans1.cluster_centers_[0],kmeans1.cluster_centers_[1])
            if threshold==0:
                threshold = 1
            a=y_pred_e[j,0:88].copy()
            a[a>=threshold]=1
            a[a<threshold]=0
            y_note_pred_e[j,:]=a
            
    
    track    = 0
    channel  = 0
    time     = 0    # In beats
    duration = 1    # In beats
    tempo    = 60   # In BPM
    volume   = 100  # 0-127, as per the MIDI standard
    MyMIDI_e = MIDIFile(1)
    MyMIDI_e.addTempo(track, time, tempo)
    
    b=postprocess(y_note_pred_e,tbin)
    b[:,0]=b[:,0]+21
    b=b[1:b.shape[0],:]
    for j in range(b.shape[0]):
        MyMIDI_e.addNote(track,channel,b[j,0].astype(np.int),b[j,1],b[j,2]*time_len,volume)
        
    with open("out_"+input_file[i][:-4]+"_e.mid", "wb") as output_file:
        MyMIDI_e.writeFile(output_file)
        
        
    threshold_pred_h = y_pred_h.copy()[:,0:88]
    num_pred_prob = y_pred_h.copy()
    
    for j in range(y_pred_h.shape[0]):
        a = num_pred_prob[:,88:num_pred_prob.shape[1]]
        num_pred=count_notes_hot.columns[np.argmax(a[j,:])]
        pred_idx = np.flip(np.argsort(threshold_pred_h[j,:]))[0:num_pred]
        b=np.zeros(88)
        if (num_pred==0):
            threshold_pred_h[j,:]=b
        else:
            b[pred_idx]=1
            threshold_pred_h[j,:]=b
        
    y_note_pred_h = threshold_pred_h[:,0:88]
    
    track    = 0
    channel  = 0
    time     = 0    # In beats
    duration = 1    # In beats
    tempo    = 60   # In BPM
    volume   = 100  # 0-127, as per the MIDI standard
    MyMIDI_h = MIDIFile(1)
    MyMIDI_h.addTempo(track, time, tempo)
    
    b=postprocess(y_note_pred_h,tbin)
    
    b[:,0]=b[:,0]+21
    b=b[1:b.shape[0],:]
    for j in range(b.shape[0]):
        MyMIDI_h.addNote(track,channel,b[j,0].astype(np.int),b[j,1],b[j,2]*time_len,volume)
        
    with open("out_"+input_file[i][:-4]+"_h.mid", "wb") as output_file:
        MyMIDI_h.writeFile(output_file)
    print('Done with', input_file[i][:-4])
    
print('DONE')

Done with African Ripples
Done with Ain't Misbehavin'
Done with Viper's Drag
Done with After You've Gone
Done with Begin the Beguine
Done with Blue And Sentimental
Done with Easy Living
Done with Fur Elise
Done with It's Always You
Done with MozartK265
Done with Scale
Done with Stardust
DONE
